**Подключаем гугл диск с данными**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/IntroduceInDataScience/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/IntroduceInDataScience


**Импортируем нужные библиотеки**

In [ ]:
#!pip install mlxtend
#!pip install pickle

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.evaluate import bias_variance_decomp
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.decomposition import PCA

**Считываем данные**

In [ ]:
data = pd.read_csv('PS_20174392719_1491204439457_log.csv')
data.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0,0


**Проверяем есть ли столбцы с пропусками** \\
Как оказалось таких нет

In [ ]:
data.columns[data.isna().any()].tolist()

[]

**Откидываем неинформативные столбцы**

In [ ]:
data.drop(['isFlaggedFraud','nameOrig','nameDest', 'amount'], axis = 1, inplace = True)
data

,step,type,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,170136.00,160296.36,0.00,0.00,0
1,1,PAYMENT,21249.00,19384.72,0.00,0.00,0
2,1,TRANSFER,181.00,0.00,0.00,0.00,1
3,1,CASH_OUT,181.00,0.00,21182.00,0.00,1
4,1,PAYMENT,41554.00,29885.86,0.00,0.00,0
...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,0.00,0.00,339682.13,1
6362616,743,TRANSFER,6311409.28,0.00,0.00,0.00,1
6362617,743,CASH_OUT,6311409.28,0.00,68488.84,6379898.11,1
6362618,743,TRANSFER,850002.52,0.00,0.00,0.00,1


In [ ]:
data = pd.get_dummies(data, columns = ['type'])
data

,step,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,170136.00,160296.36,0.00,0.00,0,0,0,0,1,0
1,1,21249.00,19384.72,0.00,0.00,0,0,0,0,1,0
2,1,181.00,0.00,0.00,0.00,1,0,0,0,0,1
3,1,181.00,0.00,21182.00,0.00,1,0,1,0,0,0
4,1,41554.00,29885.86,0.00,0.00,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,0.00,0.00,339682.13,1,0,1,0,0,0
6362616,743,6311409.28,0.00,0.00,0.00,1,0,0,0,0,1
6362617,743,6311409.28,0.00,68488.84,6379898.11,1,0,1,0,0,0
6362618,743,850002.52,0.00,0.00,0.00,1,0,0,0,0,1


In [ ]:
data.drop(['type_CASH_IN', 'type_CASH_OUT', 'type_DEBIT', 'type_TRANSFER'], axis = 1, inplace = True)
data

,step,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_PAYMENT
0,1,170136.00,160296.36,0.00,0.00,0,1
1,1,21249.00,19384.72,0.00,0.00,0,1
2,1,181.00,0.00,0.00,0.00,1,0
3,1,181.00,0.00,21182.00,0.00,1,0
4,1,41554.00,29885.86,0.00,0.00,0,1
...,...,...,...,...,...,...,...
6362615,743,339682.13,0.00,0.00,339682.13,1,0
6362616,743,6311409.28,0.00,0.00,0.00,1,0
6362617,743,6311409.28,0.00,68488.84,6379898.11,1,0
6362618,743,850002.52,0.00,0.00,0.00,1,0


**Балансировка классов (методом upsampling)**

In [ ]:
rat = len(data.loc[data['isFraud']==0])//len(data.loc[data['isFraud']==1])

df_1 = data.loc[data['isFraud']==1]
df_1 = df_1.loc[df_1.index.repeat(rat)]
train_n = pd.concat([data.loc[data['isFraud']==0], df_1]).sample(frac=1)

data = train_n.sample(n = 20000, random_state=1)

In [ ]:
data = train_n

**Отделим целевой столбец от матрицы признаков**

In [ ]:
X = data.drop('isFraud', axis = 1)
y = data['isFraud']

**Обучение дерева решений, без оптимизации признакового пространства, с базовыми параметрами. Оценим оценку кросс-валидации**

In [ ]:
decisionTree = DecisionTreeClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
scores = cross_val_score(decisionTree, X, y, cv=5)

scores

array([0.97475, 0.97525, 0.972  , 0.979  , 0.97775])

**Подбор параметров модели методом прямого поиска**

In [ ]:
def bruteForceParametrs(X, y):
  params_dict = {
      "depth" : [],
      "criterion" : ["gini", "entropy", "log_loss"]
  }

  for i in range(1, 25):
    params_dict['depth'].append(i)

  max_accuracy = [1, 'gini', -1, []]

  for depth in params_dict['depth']:
    for crit in params_dict['criterion']:
      model = DecisionTreeClassifier(max_depth = depth, criterion = crit)
      score = cross_val_score(model, X, y, cv=20)
      if score.mean() > max_accuracy[2]:
        max_accuracy[0] = depth; max_accuracy[1] = crit; max_accuracy[2] = score.mean();max_accuracy[3] = score
  print(max_accuracy)
  return max_accuracy

In [ ]:
max_accuracy = bruteForceParametrs(X, y)

[19, 'gini', 0.9787500000000001, array([0.975, 0.978, 0.972, 0.978, 0.98 , 0.983, 0.979, 0.983, 0.98 ,
       0.982, 0.968, 0.973, 0.987, 0.985, 0.981, 0.982, 0.982, 0.975,
       0.975, 0.977])]


In [ ]:
bestModel = DecisionTreeClassifier(max_depth = max_accuracy[0], criterion = max_accuracy[1])
bestModel.fit(X_train, y_train)

pred = bestModel.predict(X_test)
print(pred)

print(classification_report(y_test, pred))

[1 0 0 ... 1 1 1]
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      3018
           1       0.97      0.98      0.98      2982

    accuracy                           0.98      6000
   macro avg       0.98      0.98      0.98      6000
weighted avg       0.98      0.98      0.98      6000



In [ ]:
print("{0} средние ожидаемые потери \n {1} среднее отколонение \n {2} средняя дисперсия".format(*bias_variance_decomp(bestModel, np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test), loss='mse', num_rounds=50)))

0.031863333333333334 средние ожидаемые потери 
 0.015286733333333335 среднее отколонение 
 0.016576600000000004 средняя дисперсия


In [ ]:
confusion_matrix(y_test, pred)

array([[2928,   90],
       [  58, 2924]])

**Сохраним модель с помощью pickle**

In [ ]:
with open("DecisionTreeClassifier.pickle", "wb") as file:
    pickle.dump(bestModel, file)

**Сожмём признаковое пространство методом главных компонент**

In [ ]:
pca = PCA(n_components = 2)
pca.fit(X)

PCA(n_components=2)

In [ ]:
X_new = pd.DataFrame(pca.transform(X))
X_new

,0,1
0,-1.822639e+06,-3.543003e+05
1,-1.669403e+06,-5.062899e+05
2,3.502316e+06,3.464597e+06
3,-1.963338e+06,-2.385496e+05
4,3.980240e+06,6.657032e+06
...,...,...
19995,-1.843559e+06,-3.370898e+05
19996,-1.897707e+06,-2.960643e+05
19997,-1.222659e+06,6.140681e+05
19998,-1.853621e+06,-3.431491e+05


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new,y,train_size = 0.3, random_state = 42)

**Вновь обучим дерево решений**

In [ ]:
max_accuracy = bruteForceParametrs(X_new, y)

[24, 'gini', 0.8593, array([0.849, 0.865, 0.85 , 0.86 , 0.862, 0.871, 0.902, 0.849, 0.857,
       0.839, 0.841, 0.873, 0.863, 0.857, 0.854, 0.866, 0.849, 0.863,
       0.84 , 0.876])]


In [ ]:
bestModel = DecisionTreeClassifier(max_depth = max_accuracy[0], criterion = max_accuracy[1])
bestModel.fit(X_train, y_train)

pred = bestModel.predict(X_test)

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.85      0.76      0.80      7047
           1       0.78      0.86      0.82      6953

    accuracy                           0.81     14000
   macro avg       0.81      0.81      0.81     14000
weighted avg       0.81      0.81      0.81     14000



In [ ]:
print("{0} средние ожидаемые потери \n {1} среднее отколонение \n {2} средняя дисперсия".format(*bias_variance_decomp(bestModel, np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test), loss='mse', num_rounds=50)))

0.20457428571428576 средние ожидаемые потери 
 0.11360914285714285 среднее отколонение 
 0.09096514285714284 средняя дисперсия


**Попытка обучить нейронку**

In [ ]:
import torch
from torch import nn

In [ ]:
class Model(nn.Module):

  def __init__(self):
    super(Model, self).__init__()
    self.linear1 = nn.Linear(
        in_features = 6,
        out_features = 30
    )
    self.softmax1 = nn.Softmax(dim=1)
    self.linear2 = nn.Linear(
        in_features = 30,
        out_features = 2
    )
    self.softmax2 = nn.Softmax(dim=1)

  def forward(self, inputs_):
    x = self.linear1(inputs_)
    x = self.softmax1(x)
    x = self.linear2(x)
    x = self.softmax2(x)
    return x

In [ ]:
def train():
  from tqdm import trange
  model = Model()
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

  from torch.optim import Adam
  loss_function = nn.CrossEntropyLoss()

  optimizer = Adam(model.parameters(), lr = 0.0003)

  y_train_new = []

  for i in y_train:
    if(i == 1):
      y_train_new.append([1.0, 0])
    else:
      y_train_new.append([0, 1.0])

  for i in trange(3):
    model.zero_grad()
    x_tensor = torch.tensor(np.array(X_train), dtype = torch.float)

    ans = model(x_tensor)

    yTrain = torch.tensor(np.array(y_train_new), dtype = torch.float)
    loss = loss_function(ans, yTrain)

    loss.backward()
    optimizer.step()
  torch.save(model.state_dict(), "model1")

train()


100%|██████████| 3/3 [00:19<00:00,  6.46s/it]


In [ ]:
with torch.no_grad():
  model = Model()
  model.load_state_dict(torch.load("model1"))

  pred = []
  for i in range(len(X_test)):
    xTest = torch.tensor([np.array(X_test)[i]], dtype = torch.float)
    predict = model(xTest)
    if(i == 0):
      print(predict)
      print(torch.argmax(model(xTest)[0]))
    pred.append(0 if (torch.argmax(model(xTest)[0]) == 1) else 1)


print(np.array(pred))

from sklearn.metrics import classification_report

print(classification_report(y_test, pred))

tensor([[0.5335, 0.4665]])
tensor(0)


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pred)

array([[5876, 1144],
       [1152, 5828]])